In [38]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import pylab as plt
import seaborn as sns

%matplotlib inline   

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

from sklearn.linear_model import LogisticRegression            # clasificar, logistic regression model   

from sklearn.preprocessing import StandardScaler               # standarized             

from sklearn.model_selection import train_test_split as tts  

from sklearn.metrics import mean_squared_error as mse 
from sklearn.linear_model import LinearRegression as LinReg

from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet   # regularizacion L1+L2
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR

from xgboost import XGBRegressor as XGBR

from catboost import CatBoostRegressor as CTR

from lightgbm import LGBMRegressor as LGBMR
from sklearn.cluster import DBSCAN


ModuleNotFoundError: No module named 'h2o'

In [2]:
df=pd.read_csv('../data/salaries_data.csv')

df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [3]:
residence = []
for i in df.employee_residence:
    if i not in ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']:
        residence.append('OTHER_COUNTRY')
    else:
        residence.append(i)
df.employee_residence=residence

In [4]:
df.employment_type.value_counts()

FT    484
PT      8
CT      4
FL      4
Name: employment_type, dtype: int64

In [5]:
df.employment_type=LabelEncoder().fit_transform(df.employment_type)

In [6]:
df.employee_residence=LabelEncoder().fit_transform(df.employee_residence)

In [7]:
df=pd.get_dummies(df, columns=['work_year'], drop_first=True)
df.head()

,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,work_year_2021,work_year_2022
0,SE,2,Data Engineer,140250,USD,140250,8,100,US,M,0,1
1,SE,2,Data Engineer,135000,USD,135000,8,100,US,M,0,1
2,MI,2,BI Data Analyst,100000,USD,100000,8,100,US,M,1,0
3,MI,0,ML Engineer,270000,USD,270000,8,100,US,L,1,0
4,MI,2,Data Engineer,22000,EUR,26005,7,0,US,L,1,0


In [8]:
df=pd.get_dummies(df, columns=['experience_level'], drop_first=True)
df.head()

,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE
0,2,Data Engineer,140250,USD,140250,8,100,US,M,0,1,0,0,1
1,2,Data Engineer,135000,USD,135000,8,100,US,M,0,1,0,0,1
2,2,BI Data Analyst,100000,USD,100000,8,100,US,M,1,0,0,1,0
3,0,ML Engineer,270000,USD,270000,8,100,US,L,1,0,0,1,0
4,2,Data Engineer,22000,EUR,26005,7,0,US,L,1,0,0,1,0


In [9]:
df=pd.get_dummies(df, columns=['remote_ratio'])
df.head()

,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,company_location,company_size,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE,remote_ratio_0,remote_ratio_50,remote_ratio_100
0,2,Data Engineer,140250,USD,140250,8,US,M,0,1,0,0,1,0,0,1
1,2,Data Engineer,135000,USD,135000,8,US,M,0,1,0,0,1,0,0,1
2,2,BI Data Analyst,100000,USD,100000,8,US,M,1,0,0,1,0,0,0,1
3,0,ML Engineer,270000,USD,270000,8,US,L,1,0,0,1,0,0,0,1
4,2,Data Engineer,22000,EUR,26005,7,US,L,1,0,0,1,0,1,0,0


In [10]:
countries = []
for i in df.company_location:
    if i not in ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']:
        countries.append('OTHER_COUNTRY')
    else:
        countries.append(i)
df.company_location=countries

In [11]:
df.company_location.value_counts()

US               296
OTHER_COUNTRY     62
GB                41
CA                25
DE                23
IN                19
FR                12
ES                12
GR                10
Name: company_location, dtype: int64

In [12]:
df.job_title=LabelEncoder().fit_transform(df.job_title)

In [13]:
df.salary_currency.value_counts()

USD    329
EUR     78
GBP     38
INR     23
CAD     16
JPY      3
TRY      3
AUD      2
PLN      2
MXN      1
CHF      1
CLP      1
BRL      1
HUF      1
SGD      1
Name: salary_currency, dtype: int64

In [14]:
df.salary_currency=LabelEncoder().fit_transform(df. salary_currency)

In [15]:
df.salary_currency.value_counts()

14    329
5      78
6      38
8      23
2      16
9       3
13      3
0       2
11      2
10      1
3       1
4       1
1       1
7       1
12      1
Name: salary_currency, dtype: int64

In [16]:
df.company_location=LabelEncoder().fit_transform(df.company_location)

In [17]:
df=pd.get_dummies(df, columns=['company_size'], drop_first=True)
df.head()

,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,company_location,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE,remote_ratio_0,remote_ratio_50,remote_ratio_100,company_size_M,company_size_S
0,2,17,140250,14,140250,8,8,0,1,0,0,1,0,0,1,1,0
1,2,17,135000,14,135000,8,8,0,1,0,0,1,0,0,1,1,0
2,2,5,100000,14,100000,8,8,1,0,0,1,0,0,0,1,1,0
3,0,36,270000,14,270000,8,8,1,0,0,1,0,0,0,1,0,0
4,2,17,22000,5,26005,7,8,1,0,0,1,0,1,0,0,0,0


In [18]:
df.columns

Index(['employment_type', 'job_title', 'salary', 'salary_currency',
       'salary_in_usd', 'employee_residence', 'company_location',
       'work_year_2021', 'work_year_2022', 'experience_level_EX',
       'experience_level_MI', 'experience_level_SE', 'remote_ratio_0',
       'remote_ratio_50', 'remote_ratio_100', 'company_size_M',
       'company_size_S'],
      dtype='object')

In [19]:
X=df.drop(['salary', 'salary_in_usd'], axis=1)  # separacion X y

y=df.salary_in_usd

In [20]:
X.shape, y.shape

((500, 15), (500,))

In [21]:
X_train, X_test, y_train, y_test=tts(X, y, random_state=42)

In [22]:
#Inicialización
logreg=LogisticRegression(max_iter=2000)
linreg=LinReg()
lasso=Lasso()
ridge=Ridge()
elastic=ElasticNet()
svr=SVR()
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
lgbmr=LGBMR()
dbscan=DBSCAN()

In [23]:
#Entrenamiento
logreg.fit(X_train, y_train)
linreg.fit(X_train, y_train)
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic.fit(X_train, y_train)
svr.fit(X_train, y_train)
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)


Learning rate set to 0.035065
0:	learn: 70579.5247269	total: 141ms	remaining: 2m 20s
1:	learn: 69640.8383460	total: 142ms	remaining: 1m 10s
2:	learn: 68747.3867898	total: 143ms	remaining: 47.6s
3:	learn: 67867.9492154	total: 144ms	remaining: 36s
4:	learn: 67039.1892049	total: 146ms	remaining: 29s
5:	learn: 66284.6160747	total: 147ms	remaining: 24.3s
6:	learn: 65506.0203903	total: 148ms	remaining: 21s
7:	learn: 64826.2287007	total: 149ms	remaining: 18.4s
8:	learn: 64211.9991275	total: 150ms	remaining: 16.5s
9:	learn: 63504.7399580	total: 151ms	remaining: 14.9s
10:	learn: 62811.5036817	total: 152ms	remaining: 13.6s
11:	learn: 62348.4785129	total: 152ms	remaining: 12.5s
12:	learn: 61743.0848556	total: 153ms	remaining: 11.6s
13:	learn: 61151.5874704	total: 154ms	remaining: 10.8s
14:	learn: 60501.0301380	total: 155ms	remaining: 10.2s
15:	learn: 60062.5839733	total: 156ms	remaining: 9.58s
16:	learn: 59685.9658748	total: 156ms	remaining: 9.03s
17:	learn: 59100.1671761	total: 157ms	remaining: 

194:	learn: 36555.1674878	total: 335ms	remaining: 1.38s
195:	learn: 36507.6740799	total: 337ms	remaining: 1.38s
196:	learn: 36432.1313153	total: 338ms	remaining: 1.38s
197:	learn: 36387.0898697	total: 339ms	remaining: 1.37s
198:	learn: 36317.3340830	total: 340ms	remaining: 1.37s
199:	learn: 36269.0130390	total: 341ms	remaining: 1.36s
200:	learn: 36242.2781413	total: 342ms	remaining: 1.36s
201:	learn: 36183.6983460	total: 343ms	remaining: 1.36s
202:	learn: 36129.1895589	total: 344ms	remaining: 1.35s
203:	learn: 36077.0066938	total: 346ms	remaining: 1.35s
204:	learn: 36030.4936820	total: 347ms	remaining: 1.34s
205:	learn: 35983.9252603	total: 348ms	remaining: 1.34s
206:	learn: 35905.4115095	total: 349ms	remaining: 1.34s
207:	learn: 35859.6855692	total: 351ms	remaining: 1.33s
208:	learn: 35812.4636483	total: 352ms	remaining: 1.33s
209:	learn: 35762.6352231	total: 353ms	remaining: 1.33s
210:	learn: 35714.2737443	total: 354ms	remaining: 1.32s
211:	learn: 35647.7020868	total: 355ms	remaining

362:	learn: 30472.5972951	total: 501ms	remaining: 879ms
363:	learn: 30449.6231144	total: 502ms	remaining: 877ms
364:	learn: 30436.0510777	total: 503ms	remaining: 875ms
365:	learn: 30406.5704116	total: 504ms	remaining: 873ms
366:	learn: 30368.6404346	total: 505ms	remaining: 871ms
367:	learn: 30336.0393190	total: 506ms	remaining: 869ms
368:	learn: 30320.4614092	total: 507ms	remaining: 867ms
369:	learn: 30271.3034459	total: 508ms	remaining: 865ms
370:	learn: 30258.9106923	total: 509ms	remaining: 863ms
371:	learn: 30225.0624495	total: 510ms	remaining: 861ms
372:	learn: 30183.8847303	total: 511ms	remaining: 859ms
373:	learn: 30170.4134229	total: 512ms	remaining: 857ms
374:	learn: 30123.0654133	total: 513ms	remaining: 855ms
375:	learn: 30108.5525120	total: 514ms	remaining: 853ms
376:	learn: 30060.7785071	total: 515ms	remaining: 852ms
377:	learn: 30035.2506887	total: 516ms	remaining: 850ms
378:	learn: 30019.2393216	total: 517ms	remaining: 848ms
379:	learn: 29994.8035907	total: 519ms	remaining

533:	learn: 26978.0191543	total: 666ms	remaining: 582ms
534:	learn: 26954.1540472	total: 668ms	remaining: 580ms
535:	learn: 26949.6985037	total: 669ms	remaining: 579ms
536:	learn: 26943.8607898	total: 670ms	remaining: 577ms
537:	learn: 26937.3280307	total: 671ms	remaining: 576ms
538:	learn: 26935.7557503	total: 671ms	remaining: 574ms
539:	learn: 26910.1291121	total: 672ms	remaining: 573ms
540:	learn: 26889.5395850	total: 673ms	remaining: 571ms
541:	learn: 26860.5086941	total: 674ms	remaining: 570ms
542:	learn: 26840.8552141	total: 676ms	remaining: 569ms
543:	learn: 26822.8333654	total: 676ms	remaining: 567ms
544:	learn: 26812.1150562	total: 677ms	remaining: 566ms
545:	learn: 26800.1328073	total: 679ms	remaining: 564ms
546:	learn: 26790.3967402	total: 680ms	remaining: 563ms
547:	learn: 26754.5687123	total: 681ms	remaining: 562ms
548:	learn: 26738.9411498	total: 682ms	remaining: 560ms
549:	learn: 26729.3195874	total: 683ms	remaining: 559ms
550:	learn: 26723.5244374	total: 684ms	remaining

702:	learn: 25055.3051270	total: 834ms	remaining: 352ms
703:	learn: 25035.1076750	total: 836ms	remaining: 351ms
704:	learn: 25029.6965929	total: 837ms	remaining: 350ms
705:	learn: 25020.5988623	total: 838ms	remaining: 349ms
706:	learn: 25015.4990197	total: 839ms	remaining: 348ms
707:	learn: 24993.3399760	total: 840ms	remaining: 346ms
708:	learn: 24980.7187386	total: 841ms	remaining: 345ms
709:	learn: 24974.7469976	total: 842ms	remaining: 344ms
710:	learn: 24967.2508824	total: 843ms	remaining: 343ms
711:	learn: 24965.3115444	total: 844ms	remaining: 341ms
712:	learn: 24961.2680438	total: 845ms	remaining: 340ms
713:	learn: 24952.0163012	total: 846ms	remaining: 339ms
714:	learn: 24937.5833047	total: 847ms	remaining: 338ms
715:	learn: 24934.3507148	total: 848ms	remaining: 336ms
716:	learn: 24916.6466597	total: 849ms	remaining: 335ms
717:	learn: 24911.8266713	total: 850ms	remaining: 334ms
718:	learn: 24909.0737373	total: 851ms	remaining: 333ms
719:	learn: 24900.3097330	total: 852ms	remaining

873:	learn: 23888.0957327	total: 1s	remaining: 144ms
874:	learn: 23884.9797983	total: 1s	remaining: 143ms
875:	learn: 23880.5614762	total: 1s	remaining: 142ms
876:	learn: 23877.2909476	total: 1s	remaining: 141ms
877:	learn: 23875.6554030	total: 1s	remaining: 140ms
878:	learn: 23873.4892798	total: 1.01s	remaining: 139ms
879:	learn: 23868.7266354	total: 1.01s	remaining: 137ms
880:	learn: 23859.5109339	total: 1.01s	remaining: 136ms
881:	learn: 23849.7126781	total: 1.01s	remaining: 135ms
882:	learn: 23847.8483499	total: 1.01s	remaining: 134ms
883:	learn: 23844.4589818	total: 1.01s	remaining: 133ms
884:	learn: 23842.7649655	total: 1.01s	remaining: 132ms
885:	learn: 23836.9681968	total: 1.01s	remaining: 130ms
886:	learn: 23832.7245884	total: 1.01s	remaining: 129ms
887:	learn: 23829.2911548	total: 1.01s	remaining: 128ms
888:	learn: 23828.0158225	total: 1.02s	remaining: 127ms
889:	learn: 23823.0426966	total: 1.02s	remaining: 126ms
890:	learn: 23817.7649041	total: 1.02s	remaining: 125ms
891:	le

LGBMRegressor()

In [24]:
#predicción
y_pred_logreg=logreg.predict(X_test)   
y_pred_linreg=linreg.predict(X_test) 
y_pred_lasso=lasso.predict(X_test)   
y_pred_ridge=ridge.predict(X_test)   
y_pred_elastic=elastic.predict(X_test)   
y_pred_svr=svr.predict(X_test)   
y_pred_rfr=rfr.predict(X_test)   
y_pred_etr=etr.predict(X_test)  
y_pred_gbr=gbr.predict(X_test) 
y_pred_xgbr=xgbr.predict(X_test) 
y_pred_ctr=ctr.predict(X_test) 
y_pred_lgbmr=lgbmr.predict(X_test) 

In [25]:
print(mse(y_test, y_pred_logreg, squared=False)) 
print(mse(y_test, y_pred_linreg, squared=False)) 
print(mse(y_test, y_pred_lasso, squared=False)) 
print(mse(y_test, y_pred_ridge, squared=False)) 
print(mse(y_test, y_pred_elastic, squared=False)) 
print(mse(y_test, y_pred_svr, squared=False)) 
print(mse(y_test, y_pred_rfr, squared=False)) 
print(mse(y_test, y_pred_etr, squared=False)) 
print(mse(y_test, y_pred_gbr, squared=False)) 
print(mse(y_test, y_pred_xgbr, squared=False)) 
print(mse(y_test, y_pred_ctr, squared=False)) 
print(mse(y_test, y_pred_lgbmr, squared=False)) 

75611.90880077027
49755.48866564582
49754.34614750091
49702.610145085826
52799.51630202167
63978.48118076341
50788.36592588988
73894.96334980919
57370.910850664
61967.8106217707
50736.32749615092
46191.31416387421


In [26]:
test=pd.read_csv('../data/testeo.csv')
test.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2020,SE,FT,Machine Learning Scientist,260000,USD,JP,0,JP,S
1,2020,MI,FT,Lead Data Analyst,87000,USD,US,100,US,L
2,2020,MI,FT,Data Analyst,85000,USD,US,100,US,L
3,2020,MI,FT,Machine Learning Engineer,299000,CNY,CN,0,CN,M
4,2020,MI,FT,Product Data Analyst,450000,INR,IN,100,IN,L


In [27]:
test.drop('salary', axis=1, inplace=True)

In [28]:
test.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2020,SE,FT,Machine Learning Scientist,USD,JP,0,JP,S
1,2020,MI,FT,Lead Data Analyst,USD,US,100,US,L
2,2020,MI,FT,Data Analyst,USD,US,100,US,L
3,2020,MI,FT,Machine Learning Engineer,CNY,CN,0,CN,M
4,2020,MI,FT,Product Data Analyst,INR,IN,100,IN,L


In [29]:
test=pd.get_dummies(test, columns=['experience_level'], drop_first=True)

test.employment_type=LabelEncoder().fit_transform(test.employment_type)

test.job_title=LabelEncoder().fit_transform(test.job_title)

test.salary_currency=LabelEncoder().fit_transform(test.salary_currency)

residence = []
for i in test.employee_residence:
    if i not in ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']:
        residence.append('OTHER_COUNTRY')
    else:
        residence.append(i)
test.employee_residence=residence

test.employee_residence=LabelEncoder().fit_transform(test.employee_residence)

countries = []
for i in test.company_location:
    if i not in ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']:
        countries.append('OTHER_COUNTRY')
    else:
        countries.append(i)
test.company_location=countries

test.company_location=LabelEncoder().fit_transform(test.company_location)

test=pd.get_dummies(test, columns=['company_size'], drop_first=True)

test=pd.get_dummies(test, columns=['work_year'], drop_first=True)

test=pd.get_dummies(test, columns=['remote_ratio'])
test.head()

,employment_type,job_title,salary_currency,employee_residence,company_location,experience_level_EX,experience_level_MI,experience_level_SE,company_size_M,company_size_S,work_year_2021,work_year_2022,remote_ratio_0,remote_ratio_50,remote_ratio_100
0,1,22,11,7,7,0,0,1,0,1,0,0,1,0,0
1,1,18,11,8,8,0,1,0,0,0,0,0,0,0,1
2,1,8,11,8,8,0,1,0,0,0,0,0,0,0,1
3,1,20,2,7,7,0,1,0,1,0,0,0,1,0,0
4,1,27,7,6,6,0,1,0,0,0,0,0,0,0,1


In [30]:
X.head()

,employment_type,job_title,salary_currency,employee_residence,company_location,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE,remote_ratio_0,remote_ratio_50,remote_ratio_100,company_size_M,company_size_S
0,2,17,14,8,8,0,1,0,0,1,0,0,1,1,0
1,2,17,14,8,8,0,1,0,0,1,0,0,1,1,0
2,2,5,14,8,8,1,0,0,1,0,0,0,1,1,0
3,0,36,14,8,8,1,0,0,1,0,0,0,1,0,0
4,2,17,5,7,8,1,0,0,1,0,1,0,0,0,0


In [31]:
y_pred_lgbmr=lgbmr.predict(test)   

y_pred_lgbmr[:10]

array([ 67657.26665788, 125514.02436133, 128379.50860529,  87746.80136925,
        62512.17060343,  76757.20873674,  31676.97561868,  70454.26555874,
       120409.45367712,  73737.30324539])

In [32]:
len(y_pred_lgbmr)

107

In [33]:
res=pd.read_csv('../data/muestra.csv')

In [34]:
res.head()

,id,salary_in_usd
0,0,70377
1,1,85839
2,2,63613
3,3,88561
4,4,37621


In [35]:
res.salary_in_usd=y_pred_lgbmr

In [36]:
res.head()

,id,salary_in_usd
0,0,67657.266658
1,1,125514.024361
2,2,128379.508605
3,3,87746.801369
4,4,62512.170603


In [37]:
res.to_csv('resultado1.csv',index=False)